# 🟩 1. 커스텀 데이터셋 제작         

https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html  


- 데이터로더(필수 내용)     
- 각 훈련별로 '어떻게 훈련 되고 있는가?' -> wandb, comet.ml(정확도, loss 그래프 출력)  

In [9]:
import torch #훈련
from torch.utils.data import Dataset, DataLoader #커스텀 데이터셋을 만들기 위함

import os #로컬 컴퓨터에서 데이터셋을 불러오기 위해
import cv2 # 시각화 해서 표현하기 위함

In [10]:
# __함수__ : 던더 함수
class SimpleDataset(Dataset):
    #이 클래스의 객체가 생성되면 무조건 __init__ 함수가 동작하게 됨
    #기초 데이터, 함수 동작하도록 연결
    def __init__(self, t):
        self.t = t

    #데이터셋의 길이 반환
    #파이토치가 훈련을 할 때 len() -> 배치사이즈 대비 얼마나 데이터셋이 남아있는 지 체크할 때 씀
    def __len__(self):
        return self.t

    #데이터 + 라벨 -> 한 쌍을 뽑는 데 씀
    #idx(index) = 몇 번째 데이터? 
    def __getitem__(self, idx):
        return torch.LongTensor([idx])

In [11]:
dataset = SimpleDataset(t=7)


#이 데이터셋을 훈련에서 어떻게 활용할 것인가? 
dataloader = DataLoader(dataset = dataset ,   #어떤 데이터셋을 훈련에 쓸 거니?
                        batch_size = 2, #데이터셋을 쪼개 배치로 만듦
                        shuffle = True,    #데이터셋을 섞어주는(랜덤)
                        drop_last = True)  #배치사이즈 만들고 남은 데이터 쓸거야?

In [12]:
for e in range(3):
    print(f"epoch : {e}")

    for batch in dataloader:
        print(batch)
        print(type(batch))

epoch : 0
tensor([[6],
        [3]])
<class 'torch.Tensor'>
tensor([[5],
        [2]])
<class 'torch.Tensor'>
tensor([[0],
        [1]])
<class 'torch.Tensor'>
epoch : 1
tensor([[3],
        [6]])
<class 'torch.Tensor'>
tensor([[4],
        [1]])
<class 'torch.Tensor'>
tensor([[2],
        [0]])
<class 'torch.Tensor'>
epoch : 2
tensor([[5],
        [2]])
<class 'torch.Tensor'>
tensor([[1],
        [6]])
<class 'torch.Tensor'>
tensor([[4],
        [3]])
<class 'torch.Tensor'>


<br>

---

## 🟢 이미지 데이터셋을 커스텀화 함          

- 이미지(0~255) 숫자로 이루어진 데이터 -> 딥러닝 -> 텐서 형태로 표현  
- 텐서 형태로 이미지를 표현해 줘야 함.  

In [ ]:
# torchvision: PyTorch에서 이미지 처리/데이터셋 관련 기능을 제공하는 라이브러리
# transforms: 이미지를 딥러닝 학습에 적합한 형태로 바꿔주는 기능 모음 (자르기, 회전, 텐서 변환, 정규화 등)
# torchvision -> 딥러닝(이미지) 수행하는 클래스 이름 #transform 이미지를 조절
from torchvision import transforms  

from PIL import Image # -> 이미지를 표현할 수 있는 파이썬 라이브러리 
import matplotlib.pyplot as plt

In [ ]:
class CustomData(Dataset):
    def __init__(self, path, transform):
        self.image_path = path  # 전체 이미지 경로를 저장
        # self.image_path는 보통 문자열 경로들의 리스트(예: ["data/a.jpg","data/b.jpg", ...])
        # 또는 pathlib.Path 객체들의 리스트로 기대됩니다. "path"라는 이름 때문에 폴더 경로 하나를 받을 것처럼 보일 수 있으니
        # 실제로는 '파일 경로 리스트'인지 '폴더 경로(내부 탐색 필요)'인지 명확히 해야 합니다.
        
        self.transform = transform  # 이미지에 대해 일괄적으로 적용할 '전처리'
        # transform은 callable (예: torchvision.transforms.Compose([...]))이어야 하며,
        # 입력으로 PIL.Image을 받고 출력으로 torch.Tensor(또는 변환된 PIL/ndarray)를 반환하는 것이 일반적입니다.

    def __len__(self):
        # if (이미지수 == 라벨수) return 이미지 수
        return len(self.image_path)
        # __len__은 DataLoader가 전체 데이터 개수를 알기 위해 호출합니다.
        # image_path가 리스트가 아니라면(예: glob 결과가 빈 경우) 여기서 에러 또는 0을 반환할 수 있습니다.

    # 텐서 형태로 변환된 이미지를 돌려줌
    def __getitem__(self, idx):
        image = self.image_path[idx]
        # image 변수는 '파일 경로 문자열'입니다. 주석과 혼동하면 안 됩니다(이미지가 아님).
        # 예: image == "data/img_001.jpg"

        image = Image.open(image)  # 해당 경로에서 이미지를 읽어 옴
        # PIL.Image.open은 파일 핸들을 연 상태로 반환(지연 로딩)합니다.
        # 파일을 안전하게 닫으려면 with Image.open(path) as img: 같은 문법을 권장합니다.
        # 또한, 채널 통일을 위해 .convert('RGB') 혹은 .convert('L')을 호출하는 것이 안전합니다.

        if self.transform is not None:
            result = self.transform(image)
            # transform이 ToTensor 등을 포함하면 result는 torch.Tensor (C,H,W), dtype=float32, 값범위 0~1 이 됩니다.
            # 주의: transform이 PIL 이미지를 변경하고 닫지 않으므로 이미지 리소스 관리를 신경 써야 합니다.

        # csv,파일 이름 등을 이용해서 데이터에 맞는 라벨을 반환
        label = 1
        # 현재는 하드코딩된 더미 라벨(1)입니다. 실제 작업에서는:
        # - 파일명/폴더 구조에서 라벨을 파싱하거나,
        # - __init__에 labels 리스트/딕셔너리를 전달하거나,
        # - CSV 파일을 읽어 라벨 맵을 만들어 사용해야 합니다.
        # 라벨은 정수(int) 또는 torch.Tensor (예: torch.tensor(label, dtype=torch.long)) 로 반환하는 것이 일반적입니다.

        return result, label
        # 주의: 만약 self.transform이 None이면 result 변수가 정의되지 않아 NameError가 납니다.
        # 따라서 transform None 케이스를 반드시 처리해야 합니다.

In [15]:
transform = transforms.Compose([
    #전처리의 다양한 종류를 여기서 적용해준다.
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
image_list[0]

NameError: name 'image_list' is not defined

In [16]:
origin = 'C:/Users/jeong/Desktop/영등포/11.16 커스텀 데이터셋/train/'
image_list = [os.path.join(origin, x) for x in os.listdir(origin)]

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:/Users/jeong/Desktop/영등포/11.16 커스텀 데이터셋/train/'

In [ ]:
origin = 'C:/Users/jeong/Desktop/영등포/11.16 커스텀 데이터셋/train/'
os.listdir(origin)

<br>

## 🟢 이미지리스트를 내가 만든 데이터셋 클래스에 적용  

In [ ]:
dataset = CustomData(image_list, transform) #이미지에 대한 경로, 일괄적으로 적용할 전처리
dataloader = DataLoader(dataset = dataset,
                        batch_size = 25,
                        shuffle = True,
                        drop_last = False)

In [17]:
dataiter = iter(dataloader) # iterate 
print(dataiter)

batch = next(dataiter)
print(batch)

tensor([[2],
        [1]])


In [ ]:
images, labels = batch

In [ ]:
len(images), len(labels)

In [ ]:
# Matplotlib로 첫 번째 데이터 배치 출력
# 이미지가 배치 (N, C, H, W) 형태라고 가정 (예: N=25, C=3, H=224, W=224)
def imshow(img, title=None):
    """이미지 시각화를 위한 함수"""
    img = img.permute(1, 2, 0)  # (C, H, W) -> (H, W, C)
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.axis('off')

# 배치의 첫 번째 이미지 출력
plt.figure(figsize=(10, 10))
for i in range(len(images)):
    plt.subplot(5, 5, i + 1)  # 5x5 그리드 생성 (25개의 이미지)
    imshow(images[i])
    plt.title(f"Label: {labels[i].item()}")
plt.tight_layout()
plt.show()

In [ ]:
!pip install wandb

import wandb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
wandb.init(
    project="poth_resnet2",  # 프로젝트 이름
    #config.yaml
    config={
        "epochs": 5,
        "batch_size": 25,
        "learning_rate": 0.001,
        "optimizer": "Adam",
        "model": "ResNet18"
    }
)

In [ ]:
# ResNet 모델 정의
model = models.resnet18(pretrained=True)
num_classes = 2  # 데이터셋의 클래스 개수에 맞게 설정
model.fc = nn.Linear(model.fc.in_features, num_classes)

# GPU 사용 여부
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 훈련
num_epochs = wandb.config.epochs

for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)  # 데이터 GPU로 이동

        optimizer.zero_grad()  # 옵티마이저 초기화
        outputs = model(images)  # 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

        running_loss += loss.item()

        # **WandB 배치 로깅**
        wandb.log({"batch_loss": loss.item()})

    # 에포크 종료 시 평균 손실 로그 기록
    avg_loss = running_loss / len(dataloader)
    wandb.log({"epoch_loss": avg_loss, "epoch": epoch + 1})
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete!")

# 모델 저장 (선택 사항)
torch.save(model.state_dict(), "resnet18_poth_dataset.pth")
wandb.save("resnet18_poth_dataset.pth")
print("Model saved!")